In [1]:
import tensorflow as tf 
import numpy as np 

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

#  увеличиваем колитество каналов до 3
x_train = np.expand_dims(x_train, axis=-1)
x_train = np.repeat(x_train, 3, axis=-1)

# нормализуем
x_train = x_train.astype('float32') / 255

# изменяем размер изображений
x_train = tf.image.resize(x_train, [32,32]) # if we want to resize 

# обрабатываем подписи
y_train = tf.keras.utils.to_categorical(y_train , num_classes=10)

print(x_train.shape, y_train.shape)

(60000, 32, 32, 3) (60000, 10)


In [2]:
#определяем вход
input = tf.keras.Input(shape=(32,32,3))
#создаём сеть resnet50
resnet = tf.keras.applications.ResNet50(weights='imagenet',
                                        include_top = False, 
                                        input_tensor = input)
resnet.trainable = False
resnet.summary()


Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 38, 38, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 16, 16, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 16, 16, 64)   256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [3]:
# Создание модели составной сети
model = tf.keras.models.Sequential()
# Добавляем сверточные слои
model.add(resnet)
# Преобразуем двумерный массив MobileNet в одномерный
model.add(tf.keras.layers.Flatten())
# Полносвязный слой
model.add(tf.keras.layers.Dense(256, activation='relu'))
# Слой регуляризации (для предотвращения переобучения)
model.add(tf.keras.layers.Dropout(0.5))
# Кол-во классов
model.add(tf.keras.layers.Dense(10, activation='sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Functional)        (None, 1, 1, 2048)        23587712  
_________________________________________________________________
flatten (Flatten)            (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 256)               524544    
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                2570      
Total params: 24,114,826
Trainable params: 527,114
Non-trainable params: 23,587,712
_________________________________________________________________


In [4]:
#компиляция составной сети
model.compile(
          loss  = tf.keras.losses.CategoricalCrossentropy(),
          metrics = tf.keras.metrics.CategoricalAccuracy(),
          optimizer = tf.keras.optimizers.Adam())


In [5]:
#обучение
model.fit(x_train, y_train, batch_size=64, epochs=10, verbose = 1)

Epoch 1/10
938/938 [==============================] - 157s 162ms/step - loss: 0.8678 - categorical_accuracy: 0.7223
Epoch 2/10
938/938 [==============================] - 157s 168ms/step - loss: 0.5145 - categorical_accuracy: 0.8358
Epoch 3/10
938/938 [==============================] - 158s 169ms/step - loss: 0.4629 - categorical_accuracy: 0.8526
Epoch 4/10
938/938 [==============================] - 162s 173ms/step - loss: 0.4391 - categorical_accuracy: 0.8591
Epoch 5/10
938/938 [==============================] - 146s 156ms/step - loss: 0.4195 - categorical_accuracy: 0.8671
Epoch 6/10
938/938 [==============================] - 142s 151ms/step - loss: 0.4211 - categorical_accuracy: 0.8647
Epoch 7/10
938/938 [==============================] - 140s 149ms/step - loss: 0.4123 - categorical_accuracy: 0.8693
Epoch 8/10
938/938 [==============================] - 135s 144ms/step - loss: 0.3922 - categorical_accuracy: 0.8749
Epoch 9/10
938/938 [==============================] - 138s 147ms/step - 

In [6]:
#  увеличиваем колитество каналов до 3
x_test = np.expand_dims(x_test, axis=-1)
x_test = np.repeat(x_test, 3, axis=-1)

# нормализуем
x_test = x_test.astype('float32') / 255

# изменяем размер изображений
x_test = tf.image.resize(x_test, [32,32]) 

# one hot # обрабатываем подписи
y_test = tf.keras.utils.to_categorical(y_test , num_classes=10)

print(x_test.shape, y_test.shape)

(10000, 32, 32, 3) (10000, 10)


In [7]:
#проверка точности
model.evaluate(x_test,y_test, batch_size=10, verbose = 1, )

1000/1000 [==============================] - 34s 32ms/step - loss: 0.2204 - categorical_accuracy: 0.9294


[0.2204045206308365, 0.9294000267982483]